# 

This example demonstrates:
- how ExecFlow can be used to run an executable for which there has not been created a dedicated aiida-plugin 
- how to document your data (both existing and desired) with DLite models and partial pipelines
- how combined partial pipelines can be used to generate a desired input with the use of mappings (to and ontology)




<img src="demo_figures/samtale1.png" alt="drawing" width="600"/>


Eventually it will be possible to ask the OpenModel OIP to create a workflow based on the the knowledge it has from the ontologies and what data it has access to.
This is done by OntoFlow. 

The output of OntoFlow will be a workflow description and for our current work flow this will be something like this schematically:

<img src="demo_figures/full_workflow.png" alt="drawing" width="1000"/>

Each purple arrow-box is a step in the Declarative workchain.


## Declarative Workchain ##

This is the description of the workflow to be executed by ExecFlow. It contains several steps.

1. The first and the last step are steps that make use of oteapi and documentation of data to retrieve data from data sources (first step) or put data into a data sink (last step).

1. The three middle arrows are separate calculations that are intimately connected and therefore the output from one goes directly into input to the next step, without going through the interoperability layer.

The declarative workchain describes the workflow and consists of the following steps:

```yaml
steps:
  - workflow: execflow.oteapipipeline    # Run the pipeline that fetches data and generates the correct input
    ....
  - workflow: execflow.exec_wrapper      # Run the pre-processing tool, this is an executable that does not have an AiiDA plugin (Moltemplate)
    ....
  - workflow: execflow.exec_wrapper      # Run the MD simulation, this is an executable that does not have an AiiDA plugin (lammps)
    ...
  - calcjob: execflowdemo.lammps.density # Run a the post-processor, this is has an aiida plugin/calcjob
    ...
  - workflow: execflow.oteapipipeline    # Run tha pipeline that documents the generated data and sends it to a data sink
    ...
```

## Step1: Partial pipelines to document data ##

<img src="demo_figures/step1.drawio.png" alt="drawing" width="400"/>

Data is documented with the use of Partial pipelines which contain information about
1. where the data can be retrieved (path/database)
1. how it can be retrieved 
1. how it should be parsed 
1. which data model to parse it into
1. the mappings of the properties in the datamodel to the correct ontology

Steps 1.-4. are contained in an oteapi strategi that combines download and parsing:

```yaml
  - dataresource: name_of_choice
    downloadUrl: "path/to/file"
    mediaType: what_kind_of_file # This is is the mediaType that is registered in oteapi, each strategy in oteapi is registered with a "type".
    configuration:
      metadata: iri_of_datamodel
      excel_config:
        worksheet: "SimulationParameters"
        header_row: "1"
        row_from: "2"
        row_to: "2"
      label: moltemplate_input_2
```

We will first start with documenting one of our data sources, which in this specific case is just the specification of the simulation parameters, and can be found in an excel workbook called `multidata_moltemplateinput.xslx` and the work sheet `PhysicalParameters` as shown below:


![SimulationParametersExcel](demo_figures/SimulationParametersExcel.png)

Steps 1.-4. above are contained in an oteapi strategy that combines download and parsing:

```yaml
  - dataresource: load_data # name of this specific strategy with configuration, to be used locally
    downloadUrl: "file:///tmp/Public/Ontotrans2ndWorkshop/data/multidata_moltemplateinput.xlsx" 
    mediaType: application/vnd.dlite-xlsx # name of oteapi strategy (plugin) to use
    configuration:
      metadata: http://openmodel.eu/meta/0.1/MoltemplateInputSimulationParameters # name of datamodel
      excel_config: # Configuration for this specific oteapi strategy
        worksheet: "SimulationParameters"
        header_row: "1"
        row_from: "2"
        row_to: "2"
      label: moltemplate_input_2
```

The data model needs to be available and in our case it looks like this:

```yaml
{
    "uri": "http://openmodel.eu/meta/0.1/MoltemplateInputSimulationParameters",
    "description": "Entity describing input variables for the simulation",
    "dimensions": {
        "nrows": "Number of elements (i.e. number or rows)."
    },
    "properties": {
        "number_of_steps": {
            "type": "int",
            "description": "Number of steps for something.",
            "shape": ["nrows"]
        },
        "equilibration_steps": {
            "type": "int",
            "description": "Number of steps during equilibration.",
            "shape": ["nrows"]
        },
        "s_": {
            "type": "int",
            "description": "Not sure.",
            "shape": ["nrows"]
        },
        "production_steps": {
            "type": "int",
            "description": "Production steps.",
            "shape": ["nrows"]
        },
        "timestep": {
            "type": "int",
            "description": "Time step in fs.",
            "shape": ["nrows"]
        },
        "gridsize": {
            "type": "int",
            "description": "Grid size.",
            "shape": ["nrows"]
        },
       "gridspacing": {
            "type": "float",
            "description": "Grid spacing.",
            "shape": ["nrows"]
        }
    }
}
```

The above will fetch the data and parse it to a datamodel, but to reach interoperability the concepts in the datamodel need to be translated to a common language that we have agreed on.
This is done in a mapping strategy in the partial pipeline:
```yaml
 - mapping: mappings_1
    mappingType: mappings
    prefixes:
      emmo: http://emmo.info/emmo#
      map: http://emmo.info/domain-mappings#
      param: http://openmodel.eu/meta/0.1/MoltemplateInputSimulationParameters#
      demonto: http://openmodel.eu/0.1/domainonto#
    triples:
      - ["param:number_of_steps", "map:mapsTo", "demonto:Steps"]
      - ["param:equilibration_steps", "map:mapsTo", "demonto:StepsToEquilibration"]
      - ["param:s_", "map:mapsTo", "demonto:S"]
      - ["param:timestep", "map:mapsTo", "demonto:TimeStep"]
      - ["param:production_steps", "map:mapsTo", "demonto:StepsInProduction"]
      - ["param:gridsize", "map:mapsTo", "demonto:GridSize"]
      - ["param:gridspacing", "map:mapsTo", "demonto:GridSpacing"]
```
